In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
from scipy import stats
import math
import os
from sklearn.linear_model import LinearRegression
folder = os.getcwd() + '/'

# 4.5 Dummy coding and interaction - solution

## 4.5.1 Task 5

Open the data file loudness.csv. It contains 300 data points, taken at a local touristic cafe. Of each respondent, the nationality is recorded, as well as his decibels produced and drinkunits that were consumed. 
- Model the decibels with nationality, units drunk and age as predictors
- What are you conclusions?
- Add the interaction between units drunk and age to the model
- What are your conclusions?
- Add the interaction between units drunk and nationality to the model. (How many terms is this?)
- What are your conclusions?

## 4.5.2 Solution

### 4.5.2.1 Import data and take a first look

In [2]:
loudness = pd.read_csv(folder + 'data/loudness.csv', index_col = 0)

In [3]:
loudness.describe()

,decibels,age_drink,drinkunit
count,300.000000,300.000000,300.000000
mean,64.635790,26.974855,15.123333
std,6.641115,2.874025,3.948005
min,47.487140,19.992365,3.000000
25%,58.811017,24.890654,13.000000
50%,65.625285,26.919561,15.000000
75%,70.272250,28.812274,18.000000
max,78.507010,35.106378,25.000000


In [4]:
loudness.sample(3)

,decibels,age_drink,drinkunit,nat
188,69.911433,29.239617,17,dutch
33,74.231998,25.486183,19,dutch
227,71.736051,27.426394,11,dutch


### 4.5.2.2 Construct dummy variables and add to a model

In [5]:
dummy_cols = pd.get_dummies(loudness['nat'])
loudness = pd.concat([loudness, dummy_cols],axis = 1).copy()

In [6]:
print(type(dummy_cols))

<class 'pandas.core.frame.DataFrame'>


In [7]:
loudness.columns

Index(['decibels', 'age_drink', 'drinkunit', 'nat', 'belgian', 'dutch',
       'german'],
      dtype='object')

In [8]:
loudness.describe()

,decibels,age_drink,drinkunit,belgian,dutch,german
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,64.635790,26.974855,15.123333,0.316667,0.353333,0.330000
std,6.641115,2.874025,3.948005,0.465953,0.478804,0.470998
min,47.487140,19.992365,3.000000,0.000000,0.000000,0.000000
25%,58.811017,24.890654,13.000000,0.000000,0.000000,0.000000
50%,65.625285,26.919561,15.000000,0.000000,0.000000,0.000000
75%,70.272250,28.812274,18.000000,1.000000,1.000000,1.000000
max,78.507010,35.106378,25.000000,1.000000,1.000000,1.000000


In [9]:
loudness.sample(10)

,decibels,age_drink,drinkunit,nat,belgian,dutch,german
201,65.812435,27.424576,7,german,0,0,1
154,69.560624,27.979608,13,dutch,0,1,0
177,71.490921,27.764564,18,dutch,0,1,0
227,71.736051,27.426394,11,dutch,0,1,0
106,61.342707,21.828151,14,belgian,1,0,0
63,55.494231,29.933600,15,belgian,1,0,0
258,65.857496,25.874799,16,german,0,0,1
50,63.004656,23.482016,5,german,0,0,1
110,52.064836,29.185005,23,belgian,1,0,0
267,67.833920,32.353885,19,dutch,0,1,0


In [10]:
y = loudness.decibels.values
x = loudness[['belgian', 'dutch', 'drinkunit', 'age_drink']].values
myLR = LinearRegression()
myLR.fit(X = x, y = y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [11]:
print(myLR.intercept_)
print(myLR.coef_)

83.01383138961117
[-7.83850689  6.82768151 -0.1862759  -0.57428234]


In [12]:
print(round(myLR.score(x, y)*100,2))

90.63


### 4.5.2.3 Construct interaction and add to model

In [13]:
loudness['int_age_drinkunit'] = loudness['drinkunit'] * loudness['age_drink']

In [14]:
loudness.sample(10)

,decibels,age_drink,drinkunit,nat,belgian,dutch,german,int_age_drinkunit
45,59.968957,21.988120,15,belgian,1,0,0,329.821806
65,58.240798,33.597388,19,german,0,0,1,638.350372
271,47.487140,30.281456,24,belgian,1,0,0,726.754943
39,53.561276,25.220260,21,belgian,1,0,0,529.625466
230,63.229900,28.574096,7,german,0,0,1,200.018671
109,65.547387,27.651296,16,german,0,0,1,442.420738
129,73.399292,22.091488,12,dutch,0,1,0,265.097860
186,65.333221,25.601292,18,german,0,0,1,460.823262
145,66.876670,24.895682,18,german,0,0,1,448.122283
49,70.263343,25.150937,23,dutch,0,1,0,578.471545


In [15]:
y = loudness.decibels.values
x = loudness[['belgian', 'dutch', 'drinkunit', 'age_drink', 'int_age_drinkunit']].values
myLR = LinearRegression()
myLR.fit(X = x, y = y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [16]:
print(myLR.intercept_)
print(myLR.coef_)

55.75989042111867
[-7.93497385  6.83305117  1.59432521  0.4331458  -0.0657795 ]


In [17]:
print(round(myLR.score(x, y)*100,2))

91.92


In [18]:
#Add the interaction between units drunk and nationality to the model
#Note that we pick the 'german' level as reference level. 
#Picking another level as reference level will not change the quality of the model or the predictions.

loudness['i_bel'] = loudness['belgian'] * loudness['drinkunit']
loudness['i_dutch'] = loudness['dutch'] * loudness['drinkunit']
loudness['i_german'] = loudness['german'] * loudness['drinkunit']

y = loudness.decibels.values
x = loudness[['belgian', 'dutch', 'drinkunit', 'age_drink', 'int_age_drinkunit', 'i_bel', 'i_dutch']].values
myLR = LinearRegression()
myLR.fit(X = x, y = y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [19]:
print(myLR.intercept_)
print(myLR.coef_)

58.9493200854738
[-1.25370552  7.40531946  1.41777978  0.24587344 -0.05462948 -0.4344111
 -0.03819861]


In [20]:
print(round(myLR.score(x, y)*100,2))

93.05
